In [73]:
# # Byte Pair Encoder
# import re
# from collections import defaultdict
# training_data = open("family-guy.txt", "r")
# text = training_data.read()
# charset = "".join(sorted(set(text)))

# tokens = re.split(r'([\(\)\[\].:,!\s])', text)
# tokens = list(filter(lambda x: x != "", tokens))

# tokens = [list(token) for token in tokens]
# k = 80

# token_map = defaultdict(int)

# # prepare initial tokenset of purely characters
# for ch in text:
#     token_map[ch] += 1


# for _ in range(k):
#     updated_lst = []
#     bpe_table = defaultdict(int)
#     # find the highest frequency byte
#     for token in tokens:
#         if len(token) == 1:
#             continue
#         for i in range(len(token) - 1):
#             bpe_table["".join(token[i:i+2])] += 1

#     # get maximum frequency byte
#     max_bp = max(bpe_table, key=bpe_table.get)

#     for token in tokens:
#         temp_token = []
#         if len(token) == 1:
#             updated_lst.append(token)
#             continue

#         cont_flag = 0

#         # go over each character in token
#         for i in range(len(token)):
#             # avoid repeating tokens        
#             if cont_flag:
#                 cont_flag = 0
#                 continue
#             if i != len(token) - 1:
#                 pair = token[i:i+2]
#                 # print(pair)
#                 if "".join(pair) == max_bp:
#                     temp_token.append(max_bp)
#                     token_map[max_bp] += 1
#                     token_map[pair[0]] -= 1
#                     token_map[pair[1]] -= 1
#                     cont_flag = 1
#                     continue

#             # else add character
#             temp_token.append(token[i])
#             token_map[token[i]] += 1
            
#         updated_lst.append(temp_token)

#     # set tokens to updated_lst of merged token
#     tokens = updated_lst

In [74]:
# token_set = sorted(list(set([token_key for token_key in token_map if token_map[token_key] > -1])))
# print(token_set)

# save token_set to file
# with open("token_set.txt", "w") as f:
    # f.write("±".join(token_set))

# load token_set from file
with open("token_set.txt", "r") as f:
    token_set = f.read().split("±")

print(token_set)

from itertools import chain

# bpe_text_in = list(chain.from_iterable(updated_lst))

# # training_data = open("family-guy.txt", "r")
# # training_data = training_data.read()
# # charset = "".join(sorted(set(training_data)))

# def encoder(text_in):
#     pass
            

encoder = lambda text_in: [token_set.index(s) for s in text_in]
decoder = lambda indices: [token_set[index] for index in indices]
device = "mps"

# token_set
# print(token_map)
# token_set[-40:]
# encoder(bpe_text_in[])

['\n', ' ', '!', '"', '#', '$', '%', '&', "'", "'s", "'t", '(', ')', '*', ',', '-', '.', '/', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', ';', '<', '>', '?', '@', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'He', 'I', "I'", "I'm", 'J', 'K', 'L', 'Lo', 'Lois', 'M', 'N', 'O', 'Oh', 'P', 'Peter', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'We', 'X', 'Y', 'You', 'Z', '[', '\\', ']', '`', 'a', 'ac', 'ad', 'al', 'all', 'an', 'and', 'ar', 'as', 'at', 'ay', 'b', 'be', 'c', 'ch', 'd', 'do', 'e', 'ed', 'en', 'er', 'ere', 'es', 'et', 'eter', 'f', 'for', 'g', 'gh', 'ght', 'h', 'ha', 'hat', 'he', 'i', 'ic', 'id', 'in', 'ing', 'is', 'it', 'j', 'k', 'ke', 'l', 'ld', 'le', 'li', 'll', 'm', 'me', 'my', 'n', 'nd', 'now', 'o', 'of', 'om', 'ome', 'on', 'one', 'oo', 'or', 'ot', 'ou', 'out', 'ow', 'p', 'q', 'r', 're', 'ri', 's', 'se', 'st', 't', 'te', 'th', 'that', 'the', 'this', 'to', 'u', 'ust', 'ut', 'v', 've', 'ver', 'w', 'we', 'wi', 'x', 'y', 'you', 'z', '{', '}', '\xa0', '©', 'ª', '°', 'á', 'â', 'é', '

In [75]:
import torch
from random import randint
import numpy as np

# context window of 512
block_size = 380
batch_size = 64
vocab_size = len(token_set)
n_embed = 80*6
dropout = 0.2
n_heads = 6
num_blocks = 6
n_steps = 5000


# data = torch.tensor(encoder(bpe_text_in), dtype=torch.long, device=device)
# save tensor to file 
# torch.save(data, 'data.pt')
# load tensor from file
data = torch.load('data.pt')

n = int(0.9*len(data))
training_set = data[:n]
validation_set = data[n:]


def generate_batch():
        randlst = torch.randint(len(training_set) - block_size, (batch_size,))#.to(device="mps")
        batch = torch.stack([training_set[i:i+block_size] for i in randlst])#.to(device="mps")
        targets = torch.stack([training_set[i+1: i+block_size+1] for i in randlst])#.to(device="mps")
        return batch,targets

# x,y = generate_batch()


In [76]:
from torch import nn
from torch.nn import functional as F

class Head(nn.Module):
    def __init__(self, head_size):
        super().__init__()
        # print(head_size)
        self.head_size = head_size
        self.batch_qkv_matrices = nn.Linear(n_embed, head_size * n_heads * 3, bias=False) 
        self.register_buffer("tril", torch.tril(torch.ones((block_size, block_size))))
        self.dropout = nn.Dropout(dropout)
        self.kv_cache = None
    
    # def reset_cache(self):
    #     self.kv_cache = None

    def forward(self, x):
        # initially tensor of B,T,C
   
        q,k,v = self.batch_qkv_matrices(x).split(self.head_size * n_heads, dim=-1) # Now Q,K,V of dim B, T, head size * n_heads
       
        B,T,C = x.shape
        # # assert C % n_heads == 0
        T_Q = T
        T_KV = T
        # if model in inference (eval) mode then use cache
        if not self.training:
            if self.kv_cache is not None: 
                # concat new kv onto old kv cache -> update cache
                k_prev,v_prev = self.kv_cache
                # reset cache if block size reached
                if self.kv_cache[0].shape[-2] < block_size:
                    k = torch.cat((k_prev, k), dim=1)
                    v = torch.cat((v_prev, v), dim=1)
                    # trim key-value cache to block size
                    # k = k[:, -block_size:, :]
                    # v = v[:, -block_size:, :]

                # set T_Q to 1 as we have 1-d query
                T_Q = 1
            
            T_KV = k.shape[-2]            
            self.kv_cache = (k,v)

 
        k = k.view(B, T_KV, n_heads, self.head_size).transpose(1,2)
        # # print(f"K: {k.shape}")
        q = q.view(B, T_Q, n_heads, self.head_size).transpose(1,2) # Now of shape B, n_heads, T, head_size for BMM
        v = v.view(B, T_KV, n_heads, self.head_size).transpose(1,2)
   
        weight_mat = q @ k.transpose(-2, -1)

        weight_mat = weight_mat * (self.head_size ** -0.5) #

        if self.training:
            weight_mat = weight_mat.masked_fill(self.tril[:T, :T] == 0, float("-inf"))

        weight_mat = F.softmax(weight_mat, dim=-1)
        # # regularisation prevent overfitting
        weight_mat = self.dropout(weight_mat)
        # Multiply with values
        res = weight_mat @ v
        res = res.transpose(1,2) # B, n_heads, T, C --> B, T, n_heads, C   
        res = res.contiguous().view(B, T, C)
        return res
    
    # COMMENTED OUT FOR TESTING


class MHAttention(nn.Module):
    def __init__(self, num_heads, head_size):
        super().__init__()
        self.att_heads = Head(head_size=head_size) #nn.ModuleList([Head(head_size) for _ in range(num_heads)])
        self.projection = nn.Linear(n_embed, n_embed)
        self.dropout = nn.Dropout(dropout)
    def forward(self, x):
        # print(x.shape)
        res = self.att_heads(x)
        # res = torch.cat([att_head(x) for att_head in self.att_heads], dim=-1)
        res = self.dropout(self.projection(res))
        return res 



In [77]:
class Feedforward(nn.Module):
    def __init__(self, n_embed) -> None:
        super().__init__()
        scale_factor = 4
        self.ff = nn.Sequential(
            nn.Linear(n_embed, n_embed * scale_factor),
            nn.ReLU(),
            nn.Linear(n_embed * scale_factor, n_embed),
            nn.Dropout(dropout)
        )

    def forward(self, x):
        return self.ff(x)


class Block(nn.Module):
    def __init__(self, n_embed, n_heads) -> None:
        super().__init__()
        self.ff = Feedforward(n_embed)
        self.mhatt = MHAttention(n_heads, (n_embed // n_heads))
        self.layer_norm1 = nn.LayerNorm(n_embed) 
        self.layer_norm2 = nn.LayerNorm(n_embed)
    def forward(self, x):
        x = x + self.mhatt(self.layer_norm1(x))
        x = x + self.ff(self.layer_norm2(x))
        return x 
    


In [78]:
class GPT(nn.Module):
    def __init__(self):
        super().__init__()
        self.embedding_table = nn.Embedding(vocab_size, n_embed)
        self.pos_embedding_table = nn.Embedding(block_size, n_embed)
       
        # self.att = Head(n_embed)
        self.blocks = nn.Sequential(*[Block(n_embed, n_heads) for _ in range(num_blocks)])

        self.layernorm =  nn.LayerNorm(n_embed)

        self.lin1 = nn.Linear(n_embed, vocab_size)

        # self.att = MHAttention(n_heads, n_embed // n_heads)
        # self.ff = Feedforward(n_embed)
        
    def forward(self, data, target=None, time_inf=None):
        # print(data.shape)
        token_layer = self.embedding_table(data)
        # print(token_layer.shape)
        B,T = data.shape
        
        if time_inf != None:
            # pos_embed arguement must be tensor, must be single value of time_inf 
            pos_embed = self.pos_embedding_table(torch.ones(1, dtype=torch.long).to(device) * time_inf)
        else:
            pos_embed = self.pos_embedding_table(torch.arange(T).to(device))
        
        total = token_layer + pos_embed
        # print(f"INPUT {total.shape}")

        total = self.layernorm(self.blocks(total))
        logits = self.lin1(total)

        if target != None:
            B,T,C = logits.shape
            logits = logits.view(B*T, C) 
            target = target.view(B*T)
            loss = F.cross_entropy(logits, target)
            return logits, loss
        return logits, None
        

    def predict(self, data, temperature=1, num_tokens=350):
        prev_token = data

        for i in range(num_tokens):
            
            t_curr = (i+data.shape[-1]) % block_size

            logits,lossNone = self(prev_token, time_inf=t_curr)

            logits = logits[:,-1,:]
            logits = logits / temperature
            prob_dist = F.softmax(logits, dim=-1)
            prev_token = torch.multinomial(prob_dist, num_samples=1)
            print(decoder(prev_token[0].tolist())[0],end="")



In [79]:
# # training loop

# gpt = GPT().to(device)       
# gpt.train()
# optimiser = torch.optim.AdamW(gpt.parameters(), lr=3e-4)
# for steps in range(n_steps):
#     batch,target = generate_batch()
#     logits, loss = gpt(batch,target)
#     optimiser.zero_grad(set_to_none=True)
#     loss.backward()
    
#     optimiser.step() 
#     if steps % 50 == 0:
#         print(loss.item())



In [87]:
prompt = "Title: "
gpt = GPT().to(device)       
test = torch.tensor(encoder(["T", "it", "le", ":", " "]), dtype=torch.long, device=device)
# print(test)
# # # # # print(test.shape)
K = 1
test = test.unsqueeze(0).repeat(K, 1)
new_model = gpt.to(device)
new_model.load_state_dict(torch.load("family-guy-lm-BPE-C380-E80"))
new_model.eval()

# # # # # gpt.eval()
with torch.no_grad():
    print("Generating text...")
    print(prompt, end="")
    out = new_model.predict(test, num_tokens=2000)


Generating text...
Title: the Griffin has 17. That guy show on the $20
  batter of 20. (Flashback to the T Curbank Points to Lois
  Griffin's house.>
  Peter: "Ah, well there it is sexy."
  Lois: "Well, well were different Adult Cameraman, honey, I'd love you, but
  I was wonderful for Monday And Daniel Cameraman, I don't think I
  caught anything, but I didn't appreciate you from me. My mother, Peter put
  you so funny."
  Lois: "So the hell think you could make you Peter put your back!"
  Peter: "But, I just don't kind of crap!"
 Loretta: He's not going to see your walk.
  Stewie: You said that on your walk, if you don't have to
  do anything, does! I'm going to swore you throw about it.
  Lois: Peter, you better steal with the john about the other damage bar. Want
  some bar? 'Cause I have the suffer on?
  Lois: It's not good at me, but you're not a very idea for a provide back
  chews.
  Peter: Really? I don't think it is.
  Lois: Honey, I'm hurting you.
  Peter: Honey, you help me

In [81]:
print(f"Number of parameters: {sum(p.numel() for p in gpt.parameters() if p.requires_grad)}")

Number of parameters: 16981628


In [83]:
# Peter: AH! Agh! We've handicapped that crowd on
#   my power over and be a Chootian Banks. Or is cookie
#   deer you've gotten telling me your daughter's kindly but
#   I see my life will turn into a box criminal white, "Dear I will
#   have to leave a Salt Banks." And sometimes I can see my family girl
#   where it to be Hic-a-doo-la guy was on his crowd and she was on screen.
#   Peter: Wow! Wait, wait, wait. You can't leave. You end up on it.
#   Caddie: You're a nut! Hey, how could you? Oh, Stewie, what about
#   yoursel


In [84]:
# gpt.eval()
# gpt(torch.tensor([[0]]).to(device))